# ELECCION DE HIPERPARAMETROS.
Analizamos 2 opciones, para quedarmos con el que nos entregue los mejores resultados de presicion:


##00-BIBLIOTECAS.

In [12]:
import requests
from io import StringIO
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import cross_val_score as cv
from sklearn.model_selection import StratifiedKFold
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import  precision_score

In [2]:
# Enlace al archivo CSV en GitHub
github_link = "https://raw.githubusercontent.com/Martinerramuspe/02-DATA_SCIENCIE/main/01-TELECOM_CHURN_PROJECT/02-PREPROCESAMIENTO/Prepro01.csv"
# Descargar el archivo CSV desde GitHub
response = requests.get(github_link)
csv_data = StringIO(response.text)
df = pd.read_csv(csv_data, encoding='utf-8')

In [5]:
# Definimos las caracteristicas segun lo ya definido en la etapa de eleccion de modelo.
X= df[[ 'International plan', 'Voice mail plan', 'Number vmail messages',
       'Total day minutes', 'Total day charge', 'Total eve minutes',
        'Total eve charge', 'Total night minutes', 'Total night charge',
        'Total intl minutes', 'Total intl calls']]

y=df.Churn

In [6]:
# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

##01-RANDOM FOREST CON HIPERPARAMETROS PREESTABLECIDOS.
Medimos la presicion construyendo un modelo (random forest) con los hiperparametros prestablecidos.

In [7]:
# Instanciamos.
forest_model = RandomForestClassifier()

In [8]:
# Fit.
forest_model.fit(X_train, y_train)

RandomForestClassifier()

In [9]:
# Observamos los hiperparametros predefinidos por el modelo.
forest_model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [10]:
# Presicion en conjunto de entrenamiento.
y_pred_train = forest_model.predict(X_train)
# Determinamos precisión en el conjunto de entrenamiento.
precision_train = precision_score(y_train, y_pred_train, average='binary')  # 'binary' contempla el balance.
precision_train

1.0

In [16]:
# Presicion en conjunto de validacion.
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
forest_cv_scores = cv(forest_model, X_train, y_train, cv=skf)
forest_cv_scores

array([0.9275, 0.925 , 0.925 , 0.9325, 0.9225])

In [17]:
# Presicion en conjunto de testeo.
y_pred_test = forest_model.predict(X_test)
# Determinamos precisión en el conjunto de test.
precision_test = precision_score(y_test, y_pred_test, average='binary') # 'binary' contempla el balance.
precision_test

0.96875

##02-GRIDSEARCH CON RANGO DE HIPERPARAMETROS.
Medimos presicion utilizando distintos  rangos en los hiperparametros

In [18]:
# Definimos rangos para los distintos  hiperparámetros
param_grid = {
    'n_estimators': [75, 100, 150],
    'max_depth': [None, 20, 30],
    'min_samples_split': [1, 2],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True],
    'ccp_alpha': [0.0],
    'class_weight': [None],
    'criterion': ['gini'],
    'max_features': ['sqrt'],
    'max_leaf_nodes': [None],
    'max_samples': [None],
    'min_impurity_decrease': [0.0],
    'min_weight_fraction_leaf': [0.0],
    'n_jobs': [None],
    'oob_score': [False],
    'random_state': [None],
    'verbose': [0],
    'warm_start': [False]
}

In [19]:
# Instanciamos GridSearchCV
grid_search = GridSearchCV(estimator=forest_model, param_grid=param_grid, cv=5, scoring='f1')  # Utilizamos "f1" para contemplar el balance.

In [20]:
# Fit
grid_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
90 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py", line 340, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/util

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'bootstrap': [True], 'ccp_alpha': [0.0],
                         'class_weight': [None], 'criterion': ['gini'],
                         'max_depth': [None, 20, 30], 'max_features': ['sqrt'],
                         'max_leaf_nodes': [None], 'max_samples': [None],
                         'min_impurity_decrease': [0.0],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [1, 2],
                         'min_weight_fraction_leaf': [0.0],
                         'n_estimators': [75, 100, 150], 'n_jobs': [None],
                         'oob_score': [False], 'random_state': [None],
                         'verbose': [0], 'warm_start': [False]},
             scoring='f1')

In [21]:
# Observamos los mejores hiperparametros logrados en el GridSearch.
grid_search.best_params_

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 30,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 75,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [22]:
# Traemos tabla, con las distintas combinaciones y sus resultado en el cross.
resultado=pd.DataFrame(grid_search.cv_results_)
resultado.sort_values("rank_test_score") # Ordenamos de forma decreciente.

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_bootstrap,param_ccp_alpha,param_class_weight,param_criterion,param_max_depth,param_max_features,...,param_warm_start,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
27,0.334398,0.009350,0.012392,0.001040,True,0.0,None,gini,30,sqrt,...,False,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.757895,0.666667,0.652174,0.688172,0.702128,0.693407,0.036544,1
16,0.418229,0.009603,0.016694,0.002031,True,0.0,None,gini,20,sqrt,...,False,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.750000,0.695652,0.652174,0.681319,0.681319,0.692093,0.032223,2
4,0.428462,0.007539,0.014719,0.001035,True,0.0,None,gini,None,sqrt,...,False,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.731183,0.688172,0.659341,0.681319,0.695652,0.691133,0.023410,3
28,0.426002,0.019497,0.013875,0.000321,True,0.0,None,gini,30,sqrt,...,False,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.723404,0.703297,0.659341,0.659091,0.695652,0.688157,0.025310,4
17,0.707660,0.141051,0.023090,0.004953,True,0.0,None,gini,20,sqrt,...,False,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.744681,0.673913,0.644444,0.666667,0.709677,0.687876,0.035308,5
5,0.635944,0.018323,0.020807,0.002066,True,0.0,None,gini,None,sqrt,...,False,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.736842,0.673913,0.637363,0.673913,0.709677,0.686342,0.034067,6
29,0.792564,0.132119,0.025020,0.005179,True,0.0,None,gini,30,sqrt,...,False,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.744681,0.688889,0.652174,0.681319,0.659341,0.685281,0.032638,7
3,0.343454,0.026913,0.011478,0.000173,True,0.0,None,gini,None,sqrt,...,False,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.731183,0.695652,0.637363,0.681319,0.673684,0.683840,0.030498,8
23,0.620425,0.018477,0.019063,0.000730,True,0.0,None,gini,20,sqrt,...,False,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.717391,0.688172,0.622222,0.688889,0.681319,0.679599,0.031251,9
11,0.707737,0.138943,0.022157,0.005298,True,0.0,None,gini,None,sqrt,...,False,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.709677,0.703297,0.613636,0.681319,0.688172,0.679220,0.034331,10


# CONCLUSION
 Pudimos ver que el modelo con la presicion mas alta, corresponde a la que cuenta con los hiperparametros predefindos por el modelo.